In [91]:
import sys
from pathlib import Path

# Get the folder where this notebook is running
notebook_path = Path().resolve()

# Assume 'src' is at the project root, one level up from the notebook folder
project_root = notebook_path.parent

# Construct the full path to src
src_path = project_root / "src"

# Add src to sys.path if not already there
if str(src_path) not in sys.path:
    sys.path.insert(1, str(src_path))

print(f"Added {src_path} to sys.path")

Added /home/rodolfo/Documents/Rodolfo/Python_Projects/projetos_sem_espacos/churn-powerco/powerco-churn/src to sys.path


In [92]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import matplotlib.gridspec as gridspec
import numpy as np
from scipy.stats import linregress
import matplotlib.ticker as mtick
from sklearn.model_selection import train_test_split
from sklearn import set_config
from functools import reduce
import warnings


from powerco_churn.EDA.basic_data_wrangling import basic_wrangling
from powerco_churn.EDA.univariate_statistics import univariate_statistics, plot_histograms_countplots
from powerco_churn.EDA.bivariate_statistics import bivariate_stats
from powerco_churn.EDA.visualizing_bivariate_statistics import generate_bar_plot
from powerco_churn.EDA.outliers import calculate_outlier_threshold
from powerco_churn.EDA.skewness import correct_skew
from powerco_churn.EDA.date_utils import parse_and_format_dates

# Configuration

In [93]:
# plot will similiar to ggplot
plt.style.use('ggplot')

random_seed = 42

#set output of skllearn to be a pandas dataframe
set_config(transform_output = "default")


# Load Data

Two dataframes:

- client_data containing information about the clients (train and test)
- price_data containing the prices of power and energy during 2015

In [94]:
#train data
train_client_data = pd.read_csv('../data/raw/train/train_client_data.csv')

#test data
test_client_data  = pd.read_csv('../data/raw/test/test_client_data.csv')


In [95]:
#train data
train_price_data = pd.read_csv('../data/raw/train/train_price_data.csv')

#test data    
test_price_data  = pd.read_csv('../data/raw/test/test_price_data.csv')

#  Feature Engineering

## Client Data

### Date Features

- There is already a feature that that indicates how long the client has been with the energy company;
- Creating a new feature that indicates the length of the active contract;
- How long will it take for the contract to end?
- How long since the last modification?

The date features will be used to create three new features and then will be removed

In [96]:
date_features = ['date_activ', 'date_end', 'date_modif_prod', 'date_renewal']

In [97]:
for feature in date_features:
    train_client_data[feature] = train_client_data[feature].apply(parse_and_format_dates)
    test_client_data[feature] = test_client_data[feature].apply(parse_and_format_dates)

In [98]:
train_client_data['contract_length'] = (pd.to_datetime(train_client_data['date_end'])
                                    - pd.to_datetime(train_client_data['date_activ'])).dt.days

test_client_data['contract_length'] = (pd.to_datetime(test_client_data['date_end'])
                                    - pd.to_datetime(test_client_data['date_activ'])).dt.days


# a random date selected to be more recent than the last date in the dataset
reference_date = pd.to_datetime('2020-01-01')

train_client_data['days_until_end'] = (reference_date - pd.to_datetime(train_client_data['date_end'])).dt.days
train_client_data['days_since_modification'] = (reference_date - pd.to_datetime(train_client_data['date_modif_prod'])).dt.days

test_client_data['days_until_end'] = (reference_date - pd.to_datetime(test_client_data['date_end'])).dt.days
test_client_data['days_since_modification'] = (reference_date - pd.to_datetime(test_client_data['date_modif_prod'])).dt.days

In [99]:
train_client_data.drop(columns = date_features, inplace = True)
test_client_data.drop(columns = date_features, inplace = True)

## Price Data

Creating new features from the time series of the prices in the price_data dataframe. These new features are aggregations of the prices charged the clients during 2015

- mean prices;
- last price;
- the difference between the first and the last prices;
- the slope of the linear regression attached to the prices;
- standard deviation of the prices;

In [100]:
numeric_columns_price = train_price_data.select_dtypes(include='number').columns.to_list()
numeric_columns_price

['price_off_peak_var',
 'price_peak_var',
 'price_mid_peak_var',
 'price_off_peak_fix',
 'price_peak_fix',
 'price_mid_peak_fix']

### Mean Prices

Mean prices for each client

In [101]:
mean_price_train = train_price_data.groupby('id')[numeric_columns_price].mean().reset_index()
mean_price_test  = test_price_data.groupby('id')[numeric_columns_price].mean().reset_index()


mean_price_columns = ['id'] + [f'mean_{col}' for col in mean_price_train.columns[1:]]
mean_price_train.columns = mean_price_columns
mean_price_test.columns = mean_price_columns



Creating four new features:

- Mean of the energy price in the three periods (off-peak, mid-peak and peak);
- Mean of the potency price in the three periods;
- Difference between the energy prices in the peak period and off-peak periods;
- Difference between the potency price in the peak and off-peak periods.


In [105]:
def create_new_energy_potency_mean_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Create new features based on energy and potency prices.

    Parameters
    ----------
    df : pd.DataFrame
        The input DataFrame.

    Returns
    -------
    pd.DataFrame
        DataFrame with new features.
    """

    energy_features  = [feature for feature in df.columns if 'var' in feature]
    potency_features = [feature for feature in df.columns if 'fix' in feature]

    df['mean_energy_price']  = df[energy_features].mean(axis = 1)
    df['mean_potency_price'] = df[potency_features].mean(axis = 1)

    df['energy_peak_minus_offpeak']  = df['mean_price_peak_var'] - df['mean_price_off_peak_var']
    df['potency_peak_minus_offpeak'] = df['mean_price_peak_fix'] - df['mean_price_off_peak_fix']

    return df



In [106]:
mean_price_train  = create_new_energy_potency_mean_features(mean_price_train)
mean_price_test   = create_new_energy_potency_mean_features(mean_price_test)

In [107]:
mean_price_train.head()

,id,mean_price_off_peak_var,mean_price_peak_var,mean_price_mid_peak_var,mean_price_off_peak_fix,mean_price_peak_fix,mean_price_mid_peak_fix,mean_energy_price,mean_potency_price,energy_peak_minus_offpeak,potency_peak_minus_offpeak
0,0002203ffbb812588b632b9e628cc38d,0.124338,0.103794,0.07316,40.701732,24.421038,16.280694,0.100431,27.134488,-0.020545,-16.280694
1,0010bcc39e42b3c2131ed2ce55246e3c,0.181558,0.000000,0.00000,45.319710,0.000000,0.000000,0.060519,15.106570,-0.181558,-45.319710
2,00114d74e963e47177db89bc70108537,0.147926,0.000000,0.00000,44.266930,0.000000,0.000000,0.049309,14.755643,-0.147926,-44.266930
3,00184e957277eeef733a7b563fdabd06,0.147637,0.000000,0.00000,44.266930,0.000000,0.000000,0.049212,14.755643,-0.147637,-44.266930
4,001987ed9dbdab4efa274a9c7233e1f4,0.122756,0.102290,0.07303,40.647427,24.388455,16.258972,0.099359,27.098285,-0.020465,-16.258972


### Last Price

In [ ]:
def create_last_price_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Create new features based on the last values of the time series.

    Parameters
    ----------
    df : pd.DataFrame
        The input DataFrame.

    Returns
    -------
    pd.DataFrame
        DataFrame with new features.
    """    

    last_price = df.loc[df.groupby('id')['price_date'].idxmax()]
    last_price = last_price.reset_index(drop=True)
    last_price = last_price.drop(columns = ['price_date'])

    last_price.columns = ['id'] + [f'last_{col}' for col in last_price.columns[1:]] 

    return last_price

In [109]:
last_price_train = create_last_price_features(train_price_data)
last_price_test  = create_last_price_features(test_price_data)

# Difference between first and last prices

In [114]:
def calculate_difference_last_first_prices(df: pd.DataFrame) -> pd.DataFrame:
    """
    Create  new features that represents the difference between the last and first prices.

    Parameters
    ----------
    df : pd.DataFrame
        The input DataFrame.

    Returns
    -------
    pd.DataFrame
        DataFrame with new feature.
    """

    last = df.loc[df.groupby('id')['price_date'].idxmax()].set_index('id', drop = True)
    last = last.drop(columns = ['price_date'])
    first = df.loc[df.groupby('id')['price_date'].idxmin()].set_index('id', drop = True)
    first = first.drop(columns = ['price_date'])

    difference = last - first


    return difference

In [115]:
difference_last_first_prices_train  = calculate_difference_last_first_prices(train_price_data)
difference_last_first_prices_train  = calculate_difference_last_first_prices(test_price_data)

## Data Preprocessing

In [10]:
uni_stats_client_data = univariate_statistics(train_client_data)
uni_stats_client_data


,type,count,missing,unique,mode,min_value,q_1,median,q_3,max_value,mean,std,skew,kurtosis
feature,,,,,,,,,,,,,,
id,object,11684,0,11684,0002203ffbb812588b632b9e628cc38d,-,-,-,-,-,-,-,-,-
channel_sales,object,11684,0,8,foosdfpfkusacimwkcsosbicdxkicaua,-,-,-,-,-,-,-,-,-
cons_12m,int64,11684,0,9219,0,0,5609.0,14139.0,41121.5,6207104,159429.231,571752.758,6.029,43.573
cons_gas_12m,int64,11684,0,1716,0,0,0.0,0.0,0.0,4154590,27843.623,163052.758,10.009,139.96
cons_last_month,int64,11684,0,4225,0,0,0.0,813.0,3435.0,771203,16149.339,64088.349,6.378,47.921
forecast_cons_12m,float64,11684,0,11222,0.0,0.0,488.34,1099.43,2404.1825,82902.83,1868.707,2419.608,7.775,169.078
forecast_cons_year,int64,11684,0,3739,0,0,0.0,321.0,1769.0,175375,1426.479,3414.982,17.364,665.216
forecast_discount_energy,float64,11684,0,12,0.0,0.0,0.0,0.0,0.0,30.0,0.957,5.075,5.179,25.116
forecast_meter_rent_12m,float64,11684,0,3145,0.0,0.0,16.18,18.725,131.02,599.31,63.06,65.964,1.438,3.802


### Missing Values

In [11]:
#Trere are not missing values
train_client_data.isna().sum()

id                                0
channel_sales                     0
cons_12m                          0
cons_gas_12m                      0
cons_last_month                   0
forecast_cons_12m                 0
forecast_cons_year                0
forecast_discount_energy          0
forecast_meter_rent_12m           0
forecast_price_energy_off_peak    0
forecast_price_energy_peak        0
forecast_price_pow_off_peak       0
has_gas                           0
imp_cons                          0
margin_gross_pow_ele              0
margin_net_pow_ele                0
nb_prod_act                       0
net_margin                        0
num_years_antig                   0
origin_up                         0
pow_max                           0
churn                             0
contract_length                   0
days_until_end                    0
days_since_modification           0
dtype: int64

### Outliers